In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import StandardScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
%pip install lightgbm

/var/folders/jh/zgqt24qj3_n6m12h05s8spkr0000gn/T/ipykernel_67092/1249265048.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Note: you may need to restart the kernel to use updated packages.


In [2]:
random_state=42
best_models = {}

In [3]:
# Read in the data
# Read in the data
df = pd.read_csv('../Data/Final_skin_cancer.csv')

df.drop('drink', axis=1, inplace=True)

# New Section

In [4]:
from sklearn.model_selection import train_test_split

def splitting_data(df, sampling, test_size=0.2, random_state=123):
    # First, split the data into features and target variable
    X = df.drop(['diagnostic'], axis=1)
    y = df['diagnostic']
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Apply sampling methods to the training data based on the specified method
    if sampling == 'none':
        return X_train, X_test, y_train, y_test
    elif sampling == 'SMOTEENN':
        from imblearn.combine import SMOTEENN
        smote_enn = SMOTEENN(random_state=random_state)
        X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
        return X_resampled, X_test, y_resampled, y_test
    elif sampling == 'SMOTE':
        from imblearn.over_sampling import SMOTE
        smote = SMOTE(random_state=random_state)
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
        return X_resampled, X_test, y_resampled, y_test
    elif sampling == 'under':
        from imblearn.under_sampling import RandomUnderSampler
        rus = RandomUnderSampler(random_state=random_state)
        X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
        return X_resampled, X_test, y_resampled, y_test
    elif sampling == 'over':
        from imblearn.over_sampling import RandomOverSampler
        rus = RandomOverSampler(random_state=random_state)
        X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
        return X_resampled, X_test, y_resampled, y_test

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

def training(X_train, y_train):
    # Create a KNN classifier with 5 neighbors
    LGBM = ExtraTreesClassifier()
    # Fit the classifier to the data
    LGBM.fit(X_train, y_train)
    return LGBM

In [6]:
def best_model(modelName, accuracy, precision, recall, f1):
    best_models[modelName] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [7]:
def predict(modleName,LGBM, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = LGBM.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cr=classification_report(y_test, y_pred, output_dict=True)
    precision = cr['weighted avg']['precision']
    recall = cr['weighted avg']['recall']
    f1 = cr['weighted avg']['f1-score']
    best_model(modleName,accuracy,precision,recall,f1)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def optimize_with_grid(X_train, y_train):
    # Define a pipeline
    # Note: Scaling might not be necessary for tree-based models, but included for consistency
    pipe = Pipeline([
        ('scaler', StandardScaler()),  # Optional for ExtraTrees
        ('extra_trees', ExtraTreesClassifier(random_state=random_state))
    ])

    # Define the parameter grid to search
    param_grid = {
        'extra_trees__n_estimators': [100, 200, 300],  # Number of trees in the forest
        'extra_trees__max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
        'extra_trees__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
        'extra_trees__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
        'extra_trees__max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider when looking for the best split
    }

    # Create the GridSearchCV object
    extra_trees_cv = GridSearchCV(pipe, param_grid, cv=5, verbose=1, n_jobs=-1)

    # Perform the grid search on the provided data
    extra_trees_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = extra_trees_cv.best_params_
    best_score = extra_trees_cv.best_score_
    best_estimator = extra_trees_cv.best_estimator_
    print("Best Parameters:", best_params)
    print("Best Score:", best_score)

    return best_estimator

# Example usage
# Ensure you have defined X_train, y_train, and optionally random_state before calling this function
# best_extra_trees_model = optimize_with_grid_extra_trees(X_train, y_train, random_state=42)


<h1> LGBM on original data with optimization </h1>

In [9]:
# using function with no sampling
X_train, X_test, y_train, y_test = splitting_data(df, 'none')
# Split the data into train and test sets
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y_train.value_counts())

Number of observations in each class in the training set:
diagnostic
1    1192
0     172
Name: count, dtype: int64


In [11]:
LGBM1 = training(X_train, y_train)
y_pred = predict('original',LGBM1, X_test, y_test)


              precision    recall  f1-score   support

           0       0.79      0.79      0.79        39
           1       0.97      0.97      0.97       302

    accuracy                           0.95       341
   macro avg       0.88      0.88      0.88       341
weighted avg       0.95      0.95      0.95       341



In [12]:
best_LGBM1 = optimize_with_grid(X_train, y_train)
prediction = predict('original_grid',best_LGBM1, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': 20, 'extra_trees__max_features': 'log2', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.9398971126912304
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        39
           1       0.98      0.98      0.98       302

    accuracy                           0.96       341
   macro avg       0.90      0.90      0.90       341
weighted avg       0.96      0.96      0.96       341



/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
186 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/pipeline.py", li

<h1> LGBM using SMOTE sampling </h1>

In [13]:
X_train, X_test, y_train, y_test = splitting_data(df, 'SMOTE')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y_train.value_counts())

Number of observations in each class in the training set:
diagnostic
1    1192
0    1192
Name: count, dtype: int64


In [15]:
LGBM2 =training(X_train, y_train)
y_pred = predict('SMOTE',LGBM2, X_test, y_test)

              precision    recall  f1-score   support

           0       0.77      0.85      0.80        39
           1       0.98      0.97      0.97       302

    accuracy                           0.95       341
   macro avg       0.87      0.91      0.89       341
weighted avg       0.96      0.95      0.95       341



In [16]:
best_LGBM2 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTE_grid',best_LGBM2, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'log2', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 5, 'extra_trees__n_estimators': 100}
Best Score: 0.9563826788577066
              precision    recall  f1-score   support

           0       0.80      0.82      0.81        39
           1       0.98      0.97      0.98       302

    accuracy                           0.96       341
   macro avg       0.89      0.90      0.89       341
weighted avg       0.96      0.96      0.96       341



/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
307 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/pipeline.py", li

<h1> LGBM using SMOTEENN sampling </h1>

In [17]:
X_train, X_test, y_train, y_test = splitting_data(df, 'SMOTEENN')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
print("Number of observations in each class in the set:")
print(y_train.value_counts())

Number of observations in each class in the set:
diagnostic
1    1018
0     933
Name: count, dtype: int64


In [19]:
LGBM3 =training(X_train, y_train)
y_pred = predict('SMOTEENN',LGBM3, X_test, y_test)

              precision    recall  f1-score   support

           0       0.71      0.87      0.78        39
           1       0.98      0.95      0.97       302

    accuracy                           0.94       341
   macro avg       0.85      0.91      0.87       341
weighted avg       0.95      0.94      0.95       341



In [20]:
# from joblib import dump
# dump(LGBM3,'/content/LGBM_SMOTEENN.joblib')

In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
best_LGBM3 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTEENN_grid',best_LGBM3, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'log2', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 200}
Best Score: 0.9938566463374648
              precision    recall  f1-score   support

           0       0.71      0.87      0.78        39
           1       0.98      0.95      0.97       302

    accuracy                           0.94       341
   macro avg       0.85      0.91      0.87       341
weighted avg       0.95      0.94      0.95       341



/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
386 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/pipeline.py", li

<h1> DT on Random undersampling </h1>

In [23]:
X_train, X_test, y_train, y_test = splitting_data(df, 'under')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
print("Number of observations in each class in the set:")
print(y_train.value_counts())

Number of observations in each class in the set:
diagnostic
0    172
1    172
Name: count, dtype: int64


In [25]:
LGBM4 =training(X_train, y_train)
y_pred = predict('undersampling',LGBM4, X_test, y_test)

              precision    recall  f1-score   support

           0       0.47      0.90      0.62        39
           1       0.99      0.87      0.92       302

    accuracy                           0.87       341
   macro avg       0.73      0.88      0.77       341
weighted avg       0.93      0.87      0.89       341



In [26]:
best_LGBM4 = optimize_with_grid(X_train, y_train)
prediction = predict('undersampling_grid',best_LGBM4, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': 10, 'extra_trees__max_features': 'sqrt', 'extra_trees__min_samples_leaf': 2, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.8953537936913897
              precision    recall  f1-score   support

           0       0.51      0.90      0.65        39
           1       0.99      0.89      0.93       302

    accuracy                           0.89       341
   macro avg       0.75      0.89      0.79       341
weighted avg       0.93      0.89      0.90       341



/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
259 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/pipeline.py", li

<h1> DT on Random Oversampling </h1>

In [27]:
X_train, X_test, y_train, y_test = splitting_data(df, 'over')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
print("Number of observations in each class in the set:")
print(y_train.value_counts())

Number of observations in each class in the set:
diagnostic
1    1192
0    1192
Name: count, dtype: int64


In [29]:
LGBM5 =training(X_train, y_train)
y_pred = predict('oversampling',LGBM5, X_test, y_test)

              precision    recall  f1-score   support

           0       0.77      0.85      0.80        39
           1       0.98      0.97      0.97       302

    accuracy                           0.95       341
   macro avg       0.87      0.91      0.89       341
weighted avg       0.96      0.95      0.95       341



In [30]:
best_LGBM5 = optimize_with_grid(X_train, y_train)
prediction = predict('oversampling_grid',best_LGBM5, X_test, y_test)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'extra_trees__max_depth': None, 'extra_trees__max_features': 'sqrt', 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 100}
Best Score: 0.972735760971055
              precision    recall  f1-score   support

           0       0.75      0.85      0.80        39
           1       0.98      0.96      0.97       302

    accuracy                           0.95       341
   macro avg       0.86      0.90      0.88       341
weighted avg       0.95      0.95      0.95       341



/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
284 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hneen/miniconda3/lib/python3.11/site-packages/sklearn/pipeline.py", li

In [31]:
best_model_df = pd.DataFrame.from_dict(best_models, orient='index')
best_model_df.sort_values(by='accuracy', ascending=False, inplace=True)
best_model_df

,accuracy,precision,recall,f1
original_grid,0.958944,0.958944,0.958944,0.958944
SMOTE_grid,0.956012,0.956530,0.956012,0.956254
original,0.953079,0.953079,0.953079,0.953079
SMOTE,0.953079,0.955571,0.953079,0.954067
oversampling,0.953079,0.955571,0.953079,0.954067
oversampling_grid,0.950147,0.953516,0.950147,0.951440
SMOTEENN,0.944282,0.951529,0.944282,0.946742
SMOTEENN_grid,0.944282,0.951529,0.944282,0.946742
undersampling_grid,0.888563,0.930620,0.888563,0.901128
undersampling,0.873900,0.926456,0.873900,0.889551
